In [217]:
import ccxt
import pandas as pd

def get_data(symbol, limit):
    # Connect to Binance (or another exchange)
    exchange = ccxt.binance()

    # Get OHLCV data (Open, High, Low, Close, Volume)
    symbol = f"{symbol}/USDT"
    timeframe = '15m'  # 1-hour candlesticks

    data = exchange.fetch_ohlcv(symbol, timeframe, limit=limit)

    # Convert data to DataFrame
    df = pd.DataFrame(data, columns=['timestamp', 'open', 'high', 'low', 'close', 'volume'])
    df['timestamp'] = pd.to_datetime(df['timestamp'], unit='ms')
    df.set_index('timestamp', inplace=True)

    import talib

    # Detect candlestick patterns
    df['bullish_engulfing'] = talib.CDLENGULFING(df['open'], df['high'], df['low'], df['close']) > 0
    df['bearish_engulfing'] = talib.CDLENGULFING(df['open'], df['high'], df['low'], df['close']) < 0
    df['hammer'] = talib.CDLHAMMER(df['open'], df['high'], df['low'], df['close']) > 0
    df['shooting_star'] = talib.CDLSHOOTINGSTAR(df['open'], df['high'], df['low'], df['close']) < 0

    # Define Buy/Sell Signals Based on Patterns
    df['buy_signal'] = (df['bullish_engulfing'] | df['hammer'])  # Bullish patterns -> Buy signal
    df['sell_signal'] = (df['bearish_engulfing'] | df['shooting_star'])  # Bearish patterns -> Sell signal

    import numpy as np

    df['buy_signal_position'] = [df['low'][i] if df['buy_signal'][i] else np.nan for i in range(len(df))]
    df['sell_signal_position'] = [df['high'][i] if df['sell_signal'][i] else np.nan for i in range(len(df))]

    return df

In [222]:
# Plot realtime the predictions and the candlestick chart
import mplfinance as mpf
from IPython.display import clear_output
import time

def plot_data(df):
    candle_num = 50

    # Plot only the last 100 candlesticks
    df_plot = df.iloc[-candle_num:]

    apd = [mpf.make_addplot(df_plot['buy_signal_position'], type='scatter', markersize=100, marker='^'),
        mpf.make_addplot(df_plot['sell_signal_position'], type='scatter', markersize=100, marker='v')]

    mpf.plot(df_plot,addplot=apd,figscale=1.5, type='candle', style='charles', volume=True)

# Plot the data on new window
while True:
    df = get_data("ETH",1000)
    plot_data(df)
    clear_output(wait=True)
    time.sleep(5)

KeyboardInterrupt: 